# 모델과 훈련

* 케라스에서 모델을 만드는 방법은 3가지가 존재한다.

## Sequential 모델과 함수형 API, Model 서브클래싱이 존재

### Sequential 모델

In [1]:
from tensorflow import keras
from keras import layers

# keras 내의 Sequential() 클래스 객체 model을 생성
model = keras.Sequential([layers.Dense(64, activation = 'relu'),   # 입력/은닉층 생성: output 차원 64, 활성화함수 'relu'
                         layers.Dense(10, activation = 'softmax')])  # 출력층 생성: output 차원 10, 활성화함수 'softmax'

# 객체 생성시 한번에 정의하는 상기 방법과는 다르게 레이어를 순차적으로 쌓아 네트워크를 구축
# keras 내의 Sequential() 클래스 객체 model 생성
model = keras.Sequential()

# 입력/은닉층 생성: unit 64개, 활성화함수 'relu'
model.add(layers.Dense(64, activation = 'relu'))

# 출력층 생성: unit 10개, 활성화함수 'softmax'
model.add(layers.Dense(10, activation = 'softmax'))

### 함수형 API

In [2]:
inputs = keras.Input(shape = (3, ), name = 'my_input')

# layers.Dense를 사용하여 은닉층 Dense 레이어 정의
features = layers.Dense(64, activation = 'relu')(inputs)

# layers.Dense를 사용하여 출력층 Dense 레이어 정의
outputs = layers.Dense(10, activation = 'softmax')(features)

# 생성한 모델에 inputs 변수와 outputs 변수를 인자로 전달하여 모델 설정
model = keras.Model(inputs = inputs, outputs = outputs)

### Model 서브 클래싱

In [3]:
# 상기 모델을 클래스화
# keras.Model 클래스를 상속받아 다중 입/출력을 처리하는 ANN 모델 구현
class CustomerTicketModel(keras.Model):
    
    # 생성자 정의(인자로 num_departments 필요)
    def __init__(self, num_departments):
        # 부모 클래스인 keras.Model의 생성자 호출
        super().__init__()
        # 각 레이어 정dml
        # input data를 연결하는 concat_layer 정의
        self.concat_layer = layers.Concatenate()
        # 은닉층 Dense 레이어 mixing_layer 정의
        self.mixing_layer = layers.Dense(64, activation = 'relu')
        # priority분류 출력 레이어 priority_scorer 정의
        self.priority_score = layers.Dense(1, activation = 'sigmoid')
        # department 분류 출력 레이어 department_classifier 정의
        self.department_classifier = layers.Dense(num_departments, activation = 'softmax')
        
    # 사용자 정의 함수 call정의(인자로 input data인 inputs 필요)
    def call(self, inputs):
        
        # input data를 각 변수에 할당
        title = inputs['title']
        test_body = input['text_body']
        tags = inputs['tags']
        
        # 레이어별 변수 정의
        # concat_layer 출력값을 features에 할당
        features = self.concat_layer([title, text_body, tags])
        # 상기 features를 입력받아 mixing_layer에서 출력한 값을 다시 features에 할당
        features = self.mixing_layer(features)
        # features를 입력받아 priorty_scorer에서 분류한 값을 priority에 할당
        priority = self.priority_score(features)
        # features를 입력받아 department_classifier에서 분류한 값을 department에 할당
        departments = self.departments_classifier(features)
        
        # 분류 예측 값인 priority, department 반환
        return priority, departments
    
# 모델 클래스 객체 model 생성(인자로 부서수=4개 전달)
model = CustomerTicketModel(num_departments=4)

## 기존의 워크플로우

In [4]:
from tensorflow import keras
from keras import layers
from keras.datasets import mnist

# 사용자 함수 작성
def get_mnist_model():
    # 함수형 API를 사용했으며, 레이어를 설정해준다
    # 입력 레이어 설정
    inputs = keras.Input(shape = (28*28,))
    # 은닉 레이어 설정
    features = layers.Dense(512, activation = 'relu')(inputs)
    # 드롭아웃 레이어 설정
    # 드롭아웃 레이어 : 딥러닝 모델의 오버피팅을 방지하기 위하여 사용
    # Dense 레이어 사이에 자리하며 해당 batch 동안 드롭아웃 비율(여기서는 0.5) 만큼 뉴런들을 비활성화
    
    # 출력 레이어 설정
    outputs = layers.Dense(10, activation = 'softmax')(features)
    # 입력 레이어와 출력 레이어를 다르게 전달하여 모델 정의
    model = keras.Model(inputs, outputs)
    return model

# 데이터 로드
(images, labels), (test_images, test_labels) = mnist.load_data()

# 데이터 스케일링
images = images.reshape((60000, 28 * 28)).astype('float32') / 255
test_images = test_images.reshape((10000, 28 * 28)).astype('float32') / 255

# 학습/검증 데이터 분할
train_images, val_images = images[10000:], images[:10000]
train_labels, val_labels = labels[10000:], labels[:10000]

# 사용자 함수를 사용하여 모델 생성
model = get_mnist_model()

# 모델 컴파일
model.compile(optimizer = 'rmsprop', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

# 모델 학습
model.fit(train_images, train_labels, epochs = 3, validation_data = (val_images, val_labels))

# 모델 테스트
test_metrics = model.evaluate(test_images, test_labels)

# 분류 결과 예측
predictions = model.predict(test_images)

Epoch 1/3
1563/1563 [==============================] - 9s 6ms/step - loss: 0.2191 - accuracy: 0.9350 - val_loss: 0.1175 - val_accuracy: 0.9667
Epoch 2/3
1563/1563 [==============================] - 9s 6ms/step - loss: 0.0936 - accuracy: 0.9729 - val_loss: 0.0926 - val_accuracy: 0.9730
Epoch 3/3
313/313 [==============================] - 1s 2ms/step


## 사용자 정의 지표 만들기

In [5]:
import tensorflow as tf

# keras.metrics.Metric 클래스를 상속한 RootMeanSquaredError 클래스 작성
class RootMeanSquaredError(keras.metrics.Metric):
    
    # 생성자 정의
    def __init__(self, name="rmse", **kwargs):
        super().__init__(name=name, **kwargs)
        # MSE의 합을 저장할 가중치를 설정하고 초기값을 0으로 설정
        self.mse_sum = self.add_weight(name="mse_sum", initializer="zeros")
        # 처리된 전체 샘플 수를 저장할 가중치를 설정하고 초기값을 0으로 설정
        # 가중치의 dtype은 int32
        self.total_samples = self.add_weight(
            name="total_samples", initializer="zeros", dtype="int32")

    # 사용자 정의 함수 생성
    def update_state(self, y_true, y_pred, sample_weight=None):
        # 실제 레이블을 원핫 인코딩
        # depth=tf.shape(y_pred)[1]: 원핫 인코딩된 차원은 예측 레이블의 차원과 동일하게 설정 
        y_true = tf.one_hot(y_true, depth=tf.shape(y_pred)[1])
        # (실제 레이블 - 예측 레이블)의 제곱을 모두 더하여 MSE 산출 (mse 변수에 저장) 
        mse = tf.reduce_sum(tf.square(y_true - y_pred))
        # mse_sum 변수에 계산된 MSE(mse 변수 값)를 누적
        self.mse_sum.assign_add(mse)
        # 현재 배치에 있는 샘플 수를 구하여 변수 num_samples에 저장
        num_samples = tf.shape(y_pred)[0]
        # total_samples 변수에 현재 배치의 샘플 수(num_samples 변수 값)를 누적 
        self.total_samples.assign_add(num_samples)

    # 사용자 정의 함수 생성
    def result(self):
        # 누적된 MSE 합(self.mse_sum)을 누적된 샘플 수(self.total_samples)로 나누어 평균을 구한 후, 
        # 그 결과에 제곱근을 적용하여 최종적인 RMSE 값을 계산하고 반환
        # tf.cast(self.total_samples, tf.float32): total_samples을 실수 값으로 변환하여 계산 진행
        return tf.sqrt(self.mse_sum / tf.cast(self.total_samples, tf.float32))
    
    # 사용자 정의 함수 생성
    def reset_state(self):
        # MSE 합계를 저장하는 변수(mse_sum)를 0으로 초기화
        self.mse_sum.assign(0.)
        # 처리된 샘플 수를 저장하는 변수(total_samples)를 0으로 초기화
        self.total_samples.assign(0)

## 사용자 정의 학습 모델 생성

In [6]:
# 상기 정의한 사용자 정의 함수 get_mnist_model()로 model 생성
model = get_mnist_model()

# 모델 컴파일
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy", RootMeanSquaredError()])    # 두번째 평가지표로 상기 작성한 클래스 RootMeanSquaredError() 전달

# 모델 학습
model.fit(train_images, train_labels,
          epochs=3,
          validation_data=(val_images, val_labels))

# 모델 평가
test_metrics = model.evaluate(test_images, test_labels)
test_metrics

Epoch 1/3
1563/1563 [==============================] - 10s 6ms/step - loss: 0.2164 - accuracy: 0.9356 - rmse: 7.2350 - val_loss: 0.1161 - val_accuracy: 0.9660 - val_rmse: 7.3672
Epoch 2/3
1563/1563 [==============================] - 9s 6ms/step - loss: 0.0920 - accuracy: 0.9723 - rmse: 7.3867 - val_loss: 0.0951 - val_accuracy: 0.9710 - val_rmse: 7.4029
Epoch 3/3
313/313 [==============================] - 1s 2ms/step - loss: 0.0729 - accuracy: 0.9786 - rmse: 7.4383


[0.07294325530529022, 0.978600025177002, 7.438286781311035]

## 콜백 사용

In [7]:
# 콜백(callback): 학습 과정 중 특정 이벤트가 발생했을 때 실행되는 함수

## EarlyStopping 콜백 : 정해진 에포크 동안 모니터링 지표가 향상되지 않을 때 훈련 정지
## ModelCheckpoint 콜백 : 매 에포크 끝에서 현재 가중치를 저장

# 콜백 목록을 정의하는 list 생성
callbacks_list = [    # fit()메서드의 매개변수를 사용하여 콜백의 리스트를 모델로 전달
    keras.callbacks.EarlyStopping(   # 성능 향상이 멈추면 훈련 중지
        monitor="val_accuracy",  # 검증 정확도 모니터링
        patience=2,  # 검증 정확도가 2 epoch 동안 향상되지 않으면 학습 중단 
    ),

    keras.callbacks.ModelCheckpoint(   # 매 에포크 끝에서 현재 가중치를 저장
        filepath="checkpoint_path.keras",
        monitor="val_loss",  # 검증 손실(val_loss)을 모니터링
        save_best_only=True,  # save_best_only가 True로 설정되어 있으므로 가장 낮은 검증 손실을 가진 모델을 저장
    )   # 저장은  filepath에 지정한 것 처럼 동일한 경로에 checkpoint_path.keras 이름으로 저장
]

# 위에서 작성한 사용자 정의 함수를 바탕으로 모델 생성
model = get_mnist_model()

# 모델 컴파일
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

# 모델 학습
model.fit(train_images, train_labels,
          epochs=10,
          callbacks=callbacks_list,    # callbacks 인자에 위에서 작성한 callbacks_list 전달
          validation_data=(val_images, val_labels))

Epoch 1/10
1563/1563 [==============================] - 9s 5ms/step - loss: 0.2227 - accuracy: 0.9343 - val_loss: 0.1329 - val_accuracy: 0.9595
Epoch 2/10
1563/1563 [==============================] - 9s 6ms/step - loss: 0.0936 - accuracy: 0.9718 - val_loss: 0.1056 - val_accuracy: 0.9696
Epoch 3/10
1563/1563 [==============================] - 9s 6ms/step - loss: 0.0650 - accuracy: 0.9806 - val_loss: 0.0789 - val_accuracy: 0.9764
Epoch 4/10
1563/1563 [==============================] - 9s 5ms/step - loss: 0.0467 - accuracy: 0.9865 - val_loss: 0.0808 - val_accuracy: 0.9788
Epoch 5/10
1563/1563 [==============================] - 9s 6ms/step - loss: 0.0348 - accuracy: 0.9899 - val_loss: 0.0873 - val_accuracy: 0.9775
Epoch 6/10
1563/1563 [==============================] - 9s 6ms/step - loss: 0.0265 - accuracy: 0.9917 - val_loss: 0.0932 - val_accuracy: 0.9779


In [ ]:
# keras.callbacks.ModelCheckpoint 콜백을 통해 저장된 모델 로드
model = keras.models.load_model("checkpoint_path.keras")